In [1]:
import fsspec
import ujson

fs_read = fsspec.filesystem('s3', anon=True, skip_instance_cache=True)
fs_write = fsspec.filesystem('')

In [2]:
hrr_subhourly_member_files = fs_read.glob('s3://noaa-hrrr-bdp-pds/hrrr.20230927/conus/hrrr.t00z.wrfsubhf*.grib2')

files = sorted(['s3://'+f for f in hrr_subhourly_member_files])
print(f'Read {len(files)} HRRR files')

Read 19 HRRR files


In [4]:
import ujson
from kerchunk.grib2 import scan_grib

afilter={'typeOfLevel': 'surface', 'stepType': 'instant'}
so = {"anon": True}
json_dir = 'idx/'

def make_json_name(file_url, message_number): #create a unique name for each reference file
    date = file_url.split('/')[3].split('.')[1]
    name = file_url.split('/')[5].split('.')[0:3]
    return f'{json_dir}{date}_{name[0]}_{name[1]}_{name[2]}_message{message_number}.json'

def gen_json(file_url):
    out = scan_grib(file_url, storage_options=so, filter=afilter)   #create the reference using scan_grib
    for i, message in enumerate(out): # scan_grib outputs a list containing one reference per grib message
        out_file_name = make_json_name(file_url, i)  #get name
        with fs_write.open(out_file_name, "w") as f: 
            f.write(ujson.dumps(message)) #write to file

In [7]:
gen_json(files[1]) 

In [8]:
from kerchunk.combine import MultiZarrToZarr

reference_jsons = fs_write.ls(json_dir) #get list of file names

#combine individual references into single consolidated reference
mzz = MultiZarrToZarr(reference_jsons,
                        concat_dims = ['valid_time'],
                        identical_dims=['latitude', 'longitude'])

d = mzz.translate()

with open(f'{json_dir}/idx.json', 'w') as f:
    f.write(ujson.dumps(d))

In [10]:
import xarray as xr

fs = fsspec.filesystem("reference", fo=d, remote_protocol='s3', remote_options={'anon':True})
m = fs.get_mapper("")
ds = xr.open_dataset(m, engine="zarr", backend_kwargs=dict(consolidated=False), 
                      chunks={'valid_time':1})
ds

<xarray.Dataset>
Dimensions:     (valid_time: 4, y: 1059, x: 1799, step: 1, surface: 1, time: 1)
Coordinates:
  * step        (step) timedelta64[ns] 00:15:00
  * surface     (surface) int64 0
  * time        (time) datetime64[ns] 2023-09-27
  * valid_time  (valid_time) datetime64[ns] 2023-09-27T00:15:00 ... 2023-09-2...
Dimensions without coordinates: y, x
Data variables: (12/18)
    cfrzr       (valid_time, y, x) float64 dask.array<chunksize=(1, 1059, 1799), meta=np.ndarray>
    cicep       (valid_time, y, x) float64 dask.array<chunksize=(1, 1059, 1799), meta=np.ndarray>
    cpofp       (valid_time, y, x) float64 dask.array<chunksize=(1, 1059, 1799), meta=np.ndarray>
    crain       (valid_time, y, x) float64 dask.array<chunksize=(1, 1059, 1799), meta=np.ndarray>
    csnow       (valid_time, y, x) float64 dask.array<chunksize=(1, 1059, 1799), meta=np.ndarray>
    dlwrf       (valid_time, y, x) float64 dask.array<chunksize=(1, 1059, 1799), meta=np.ndarray>
    ...          ...
    sp          (valid_time, y, x) float64 dask.array<chunksize=(1, 1059, 1799), meta=np.ndarray>
    ulwrf       (valid_time, y, x) float64 dask.array<chunksize=(1, 1059, 1799), meta=np.ndarray>
    uswrf       (valid_time, y, x) float64 dask.array<chunksize=(1, 1059, 1799), meta=np.ndarray>
    vbdsf       (valid_time, y, x) float64 dask.array<chunksize=(1, 1059, 1799), meta=np.ndarray>
    vddsf       (valid_time, y, x) float64 dask.array<chunksize=(1, 1059, 1799), meta=np.ndarray>
    vis         (valid_time, y, x) float64 dask.array<chunksize=(1, 1059, 1799), meta=np.ndarray>
Attributes:
    centre:             kwbc
    centreDescription:  US National Weather Service - NCEP
    edition:            2
    subCentre:          0

In [11]:
next_timestep_idx_files = hrr_subhourly_member_files = fs_read.glob('s3://noaa-hrrr-bdp-pds/hrrr.20230927/conus/hrrr.t01z.wrfsubhf*.grib2.idx')
idx_files = sorted(['s3://'+f for f in next_timestep_idx_files])
print(f'Read {len(idx_files)} HRRR files')

Read 19 HRRR files


In [39]:
def read_refs(file_url):
    with fs_write.open(file_url, "r") as f:
        raw = ujson.loads(f.read())
    return raw['refs']

In [41]:
refs = read_refs(f'{json_dir}/idx.json')

In [44]:
def parse_idx(file_url):
    messages = []
    with fs_read.open(file_url, 'r') as f:
        for line in f.readlines():
            parts = str(line).split(':')
            messages.append({
                'index': int(parts[0]),
                'offset': int(parts[1]),
                'date': parts[2].split('=')[1],
                'var': parts[3].lower(),
                'surface': parts[4],
                'valid_time': parts[5],
            })
    return messages

In [45]:
idx_refs = parse_idx(idx_files[1])

In [48]:
ref_vars = set([k.split('/')[0] for k in refs.keys()])
ref_vars

{'.zattrs',
 '.zgroup',
 'cfrzr',
 'cicep',
 'cpofp',
 'crain',
 'csnow',
 'dlwrf',
 'dswrf',
 'gust',
 'latitude',
 'longitude',
 'orog',
 'prate',
 'sp',
 'step',
 'surface',
 'time',
 'ulwrf',
 'uswrf',
 'valid_time',
 'vbdsf',
 'vddsf',
 'vis'}

In [54]:
matching_vars = [r for r in idx_refs if (r['var'] in ref_vars and r['surface'] == 'surface')]
f'Found {len(matching_vars)} matching variables'

'Found 64 matching variables'

Okay now that we have the matching variables, we need to get the time array for the existing references. Im not sure how to decode these yet. 

Once they are decoded, we need to match up the variables from the idx files then append the new chunk to the exisiting array using the offsets found from the idx files.

In [69]:
import base64

base64.b64decode(refs['time/0'].encode('utf-8')[7:])

b'\x80p\x13e\x00\x00\x00\x00'